In [1]:
#provisionval Btrack - notebook

import os
import json

import btrack
import napari

import numpy as np
import pandas as pd

from skimage import io
from napari.utils import nbscreenshot
import tifffile as tiff

import napari
#from napari.utils import nbscreenshot
import glob
#import numpy as np
#from skimage import io
import matplotlib.pyplot as plt



In [2]:
# Import Centroids of detected masks and then convert

points_conepi = np.genfromtxt("/home/lpe/Desktop/cellpose/epithelial_points2track.txt")    
points_flipped = np.swapaxes(points_conepi, 0, 1)
points_flippedc = np.swapaxes(points_conepi, 0, 1)

new_points = points_conepi[points_conepi[:,0] == 0,:]

objectz = btrack.dataio.objects_from_array(points_conepi)

In [3]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config.json')
    # append the objects to be tracked
    tracker.append(objectz)
    tracker.update_method = btrack.constants.BayesianUpdates.EXACT
    tracker.max_search_radius = 12

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0,1024), (0,1024), (-1e5,1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()
    
    tracks = tracker.tracks

#     tracker.export('./test3.hdf5', obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2021/03/17 04:11:27 PM] Loaded btrack: /home/lpe/miniconda3/envs/btrack/lib/python3.7/site-packages/btrack/libs/libtracker.so
[INFO][2021/03/17 04:11:27 PM] btrack (v0.4.0) library imported
[INFO][2021/03/17 04:11:27 PM] Setting max XYZ search radius to: 100
[INFO][2021/03/17 04:11:27 PM] Starting BayesianTracker session
[INFO][2021/03/17 04:11:27 PM] Loading configuration file: cell_config.json
[INFO][2021/03/17 04:11:27 PM] Loading motion model: b'cell_motion'
[INFO][2021/03/17 04:11:27 PM] Objects are of type: <class 'list'>
[INFO][2021/03/17 04:11:27 PM] Setting Bayesian update method to: BayesianUpdates.EXACT
[INFO][2021/03/17 04:11:27 PM] Setting max XYZ search radius to: 12
[INFO][2021/03/17 04:11:27 PM] Set volume to ((0, 1024), (0, 1024), (-100000.0, 100000.0))
[INFO][2021/03/17 04:11:27 PM] Starting tracking... 
[INFO][2021/03/17 04:11:27 PM] Tracking objects in frames 0 to 99 (of 282)...
[INFO][2021/03/17 04:11:32 PM]  - Timing (Bayesian updates: 58.41ms, Linking: 1.8

In [4]:
long_tracks = []

for trackelt in tracks:
    if len(trackelt) > 50:
        
        timestamps = trackelt.t
        x = trackelt.x
        y = trackelt.y
        track_id = trackelt.ID

        trackz = np.zeros(((len(trackelt)), 4), dtype=np.float32)
    
        track_id = track_id * len(trackelt)
        trackz[:, 0] = track_id
        trackz[:, 1] = timestamps
        trackz[:, 3] = y
        trackz[:, 2] = x

        long_tracks.append(trackz)
        
long_tracks = np.concatenate(long_tracks, axis=0)



In [5]:

origin=(1024/2,1024/2)
vascu_frames = "/home/lpe/Desktop/vasc_tsrs/200123_mEos_cells_4.tif"###### Our times series
vascu_tif = tiff.imread(vascu_frames)
print(vascu_tif.shape)

vascu_tif = vascu_tif[1:283,0,:,:]

with napari.gui_qt():
    viewer = napari.Viewer()
    viewer.add_tracks(long_tracks, name='fat stacks')
    viewer.add_image(vascu_tif)

#    viewer.add_points(
#            points_flipped,
#            size = .8,
#            face_color="orange",
#            symbol = 'ring'
#            )

(284, 3, 1024, 1024)
